In [16]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok

In [15]:
!pip install pyngrok
!pyngrok authtoken 29JlwqUGLZxPpC0QfilJEO8AwsW_xxCAFaGtukqQ8krP2CFd

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

In [18]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("LINE_CHANNEL_ACCESS_TOKEN") #Channel_Access_token
handler = WebhookHandler("LINE_CHANNEL_SECRET") #Channel_Secret

In [19]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [20]:

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [ ]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://91f5-35-188-204-162.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/May/2022 04:57:27] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[]}


127.0.0.1 - - [18/May/2022 04:58:19] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106246585972","text":"測試"},"webhookEventId":"01G3ARFXCTBRRN11N529BTFBAT","deliveryContext":{"isRedelivery":false},"timestamp":1652849898393,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"fc7733b64b57439c92a389edf9eea831","mode":"active"}]}
{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"follow","webhookEventId":"01G3AS24YH670NRPKXEXV2Y29S","deliveryContext":{"isRedelivery":false},"timestamp":1652850496286,"source":{"type":"user","userId":"U71fc47e1e3e0ef644d3a9492dc31f411"},"replyToken":"da389ee2ac334e6d87f965813162421a","mode":"active"}]}


127.0.0.1 - - [18/May/2022 05:08:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 05:08:30] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"unfollow","webhookEventId":"01G3AS2J3WGCF54098R1XVHW92","deliveryContext":{"isRedelivery":false},"timestamp":1652850509821,"source":{"type":"user","userId":"U71fc47e1e3e0ef644d3a9492dc31f411"},"mode":"active"}]}
